In [771]:
import pandas as pd

In [772]:
df = pd.read_excel(r'C:\Users\moham\OneDrive\Desktop\Python\DA\CRM customer\novalink_raw_data.xlsx')

In [773]:
df.info()

# Missing data? ✅
# Column datatypes correct? ⛔


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Customer ID            2966 non-null   object 
 1   Full Name              3000 non-null   object 
 2   Signup Date            3000 non-null   object 
 3   Last Transaction Date  3000 non-null   object 
 4   Plan Type              3000 non-null   object 
 5   Monthly Fee ($USD)     3000 non-null   object 
 6   Total Transactions     1984 non-null   float64
 7   Payment Method         3000 non-null   object 
 8   City                   3000 non-null   object 
 9   State                  0 non-null      float64
 10  Country                3000 non-null   object 
 11  Churned                1949 non-null   object 
 12  Support Tickets        2505 non-null   object 
 13  Rating                 3000 non-null   object 
 14  Notes                  3000 non-null   object 
dtypes: f

In [774]:
# ROW PRE-PROCESSING

# Full name
df['Full Name'] = df['Full Name'].str.replace(r'Dr.','').str.strip().str.title()

# Signup date
df['Signup Date'] = pd.to_datetime(df['Signup Date'],format='mixed', errors='coerce')
df['Signup Date'] = df['Signup Date'].dt.strftime('%Y-%m-%d')

# Transaction date
df['Last Transaction Date'] = pd.to_datetime(df['Last Transaction Date'], errors='coerce').dt.date

# Plan type
df['Plan Type'] = df['Plan Type'].str.lower().str.strip()
plan_type = {'goldd':'gold','silver':'silver', 'gold':'gold', 'bronze':'bronze',}
df['Plan Type'] = df['Plan Type'].map(plan_type).str.title()

# Monthly fee
df['Monthly Fee ($USD)'] = df['Monthly Fee ($USD)'].astype(str).str.strip()
df['Monthly Fee ($USD)'] = df['Monthly Fee ($USD)'].str.replace(r'[$]|USD','', regex=True)
df['Monthly Fee ($USD)'] = df['Monthly Fee ($USD)'].str.replace(r'\,', '.', regex=True)
df['Monthly Fee ($USD)'] = pd.to_numeric(df['Monthly Fee ($USD)'], errors='coerce')

# Payment method
df['Payment Method'] = df['Payment Method'].str.strip().str.lower()
pay_type = {'credit card':'credit card', 'cc':'credit card', 'paypal':'PayPal', 'creditcard':'credit card', 'bank transfer':'bank transfer','banktransfer':'bank transfer'}
df['Payment Method'] = df['Payment Method'].map(pay_type).str.title()

# City
df['City'] = df['City'].str.lower().str.title()

# Country
df['Country'] = df['Country'].str.strip().str.lower()
country_type = {'mexico':'mexico', 'can':'canada', 'usa':'united states', 'mex':'mexico', 'canada':'canada', 'u.s.':'united states', 'united states':'united states'}
df['Country'] = df['Country'].map(country_type).str.title()

# Churned
df['Churned'] = df['Churned'].str.strip().str.lower()
churn_type = {'y':'yes', 'n':'no', 'yes':'yes', 'no':'no'}
df['Churned'] = df['Churned'].map(churn_type).str.title()

# Rating
df['Rating'] = df['Rating'].str.strip()
ratings = {'four':4, '**':2, 'three':3, '****':4, 'five':5, '3':3, '***':3, '2':2, 'two':2, '*':1,'5':5, '1':1, '*****':5, '4':4, 'one':1}
df['Rating'] = df['Rating'].map(ratings)

# Support ticket
df['Support Tickets'] = df['Support Tickets'].str.strip()
df[['Ticket','Note']] = df['Support Tickets'].str.split(r',|-', expand=True)
df[['Ticket','Note']] = df[['Ticket','Note']].replace('None', 'N/A')
df['Ticket'] = df['Ticket'].str.replace(r'!','', regex=True).str.capitalize()
df['Note'] = df['Note'].str.strip().str.capitalize()

# NA'S
# Fill object columns
obj_col = df.select_dtypes(include=['object']).columns
df[obj_col] = df[obj_col].fillna('N/A')

# Fill numeric columns
num_col = df.select_dtypes(include=['float64']).columns
df[num_col] = df[num_col].fillna(df[num_col].median())


C:\Users\moham\AppData\Local\Temp\ipykernel_1936\1979286731.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Last Transaction Date'] = pd.to_datetime(df['Last Transaction Date'], errors='coerce').dt.date


In [775]:
# COLUMN PRE-PROCESSING

# Drop columns
df = df.drop(columns=['Customer ID', 'State', 'Support Tickets','Notes'])

# Rename columns
df = df.rename(columns={'Total Transactions':'Total Transaction ($USD)'})

# Drop NaN & duplicates
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

df

,Full Name,Signup Date,Last Transaction Date,Plan Type,Monthly Fee ($USD),Total Transaction ($USD),Payment Method,City,Country,Churned,Rating,Ticket,Note
0,Daniel Doyle,2024-06-27,2025-05-24,Silver,64.00,26.5,Credit Card,New Roberttown,Mexico,N/A,4,Connection drops frequently,Slow response time
1,Rhonda Smith,2022-04-16,2025-05-08,Gold,53.00,6.0,Credit Card,Petersonberg,Canada,Yes,2,Slow internet speed,N/A
2,Valerie Gray,2023-12-16,2024-01-12,Bronze,34.00,50.0,Paypal,Barbaraland,United States,No,3,Cannot login to account,N/A
3,Carla Kelly,2024-06-17,2024-11-21,Bronze,30.00,26.5,Credit Card,New Cynthiaside,Mexico,Yes,4,Billing issue,Charged twice
4,Ethan Adams,2022-07-04,2025-07-30,Bronze,60.92,10.0,Credit Card,Jasonfort,Canada,Yes,5,Connection drops frequently,Slow response time
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Brenda Adams,2022-12-11,2025-04-12,Silver,34.14,33.0,Credit Card,Jamesberg,United States,Yes,5,Billing issue,Charged twice
2996,Claire Maddox,2022-05-09,2025-05-15,Gold,92.29,33.0,Paypal,East Garrett,Canada,No,3,Upgrade request,N/A
2997,Mary King,2023-12-02,2025-03-24,Gold,87.00,26.5,Paypal,Amberburgh,United States,No,1,Slow internet speed,N/A
2998,Latasha Miller,2023-10-09,2025-05-11,Bronze,87.00,1.0,Paypal,Port Williamborough,Mexico,N/A,1,Cannot login to account,N/A


In [776]:
df.to_excel('test.xlsx', index=False)